## 과업
### 1. mediapipe csv 추출
* n초 동안 움직임을 시작지점 - 끝나는 지점 추출해서 그 동안만 csv로 생성하기
### 2. 추출한 csv 모델 비교 후 결과값 받기

#### 영상을 넣어서 그 영상의 joint값 추출하기 

In [1]:
import cv2
import mediapipe as mp
import pandas as pd
import time
import datetime
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
select_yoga

In [8]:
# 점과 선의 스타일 정의
dot_color = (255, 0, 0)  # 파란색 점
dot_radius = 10  # 점 크기
line_color = (255, 255, 255)  # 흰색 선
line_thickness = 2  # 선 두께

# MediaPipe 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

#현시간 문자열로
suffix = datetime.datetime.now().strftime('%y%m%d_%H%M%S')

# 랜드마크 선택: [0, 11~16, 23~28]
selected_landmarks = [0] + list(range(11, 17)) + list(range(23, 29))

# 저장 파일명
user_id = 'id123'
filepath = f'{user_id}_{suffix}' #id + 현시각
output_csv_path = f'./BPE/training/{filepath}.csv' # 최종파일이름

# 관절 ID별 컬럼 이름 생성 (x, y만 포함)
columns = ['frame'] + [f'landmark{idx}_{axis}' for idx in selected_landmarks for axis in ['x', 'y']] + \
          ['neck_x', 'neck_y', 'hip_x', 'hip_y']

# 데이터 저장용 리스트
data = []

# 카메라 열기
cap = cv2.VideoCapture(0)  # 0번 카메라
frame_idx = 0
start_time = time.time()
duration = 30  # 30초 동안 실행

while cap.isOpened() and time.time() - start_time < duration:
    ret, frame = cap.read()
    if not ret:
        break

    frame_idx += 1

    # BGR 이미지를 RGB로 변환
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # MediaPipe로 자세 인식 수행
    results = pose.process(rgb_frame)

    # 프레임 데이터 초기화
    frame_data = [frame_idx]  # 현재 프레임 번호 추가

    # 선택된 랜드마크 정보 추가
    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        for idx in selected_landmarks:
            landmark = landmarks[idx]
            frame_data.extend([landmark.x, landmark.y])

        # 목 좌표 계산 (11번과 12번의 중간값)
        left_shoulder = landmarks[11]
        right_shoulder = landmarks[12]
        neck_x = (left_shoulder.x + right_shoulder.x) / 2
        neck_y = (left_shoulder.y + right_shoulder.y) / 2
        frame_data.extend([neck_x, neck_y])

        # 엉덩이 좌표 계산 (23번과 24번의 중간값)
        left_hip = landmarks[23]
        right_hip = landmarks[24]
        hip_x = (left_hip.x + right_hip.x) / 2
        hip_y = (left_hip.y + right_hip.y) / 2
        frame_data.extend([hip_x, hip_y])

        # 시각화: 특정 랜드마크만 표시
        for idx in selected_landmarks:
            landmark = landmarks[idx]
            x, y = int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0])
            cv2.circle(frame, (x, y), dot_radius, dot_color, -1)  # 파란 점

        # 목과 엉덩이 시각화
        neck_pos = (int(neck_x * frame.shape[1]), int(neck_y * frame.shape[0]))
        hip_pos = (int(hip_x * frame.shape[1]), int(hip_y * frame.shape[0]))
        cv2.circle(frame, neck_pos, dot_radius, dot_color, -1)  # 목 (파란 점)
        cv2.circle(frame, hip_pos, dot_radius, dot_color, -1)  # 엉덩이 (파란 점)

        # 목과 어깨를 잇는 선
        cv2.line(frame, neck_pos, 
                 (int(left_shoulder.x * frame.shape[1]), int(left_shoulder.y * frame.shape[0])), 
                 line_color, line_thickness)  # 흰 선
        cv2.line(frame, neck_pos, 
                 (int(right_shoulder.x * frame.shape[1]), int(right_shoulder.y * frame.shape[0])), 
                 line_color, line_thickness)  # 흰 선

        # 엉덩이와 다리를 잇는 선
        cv2.line(frame, hip_pos, 
                 (int(left_hip.x * frame.shape[1]), int(left_hip.y * frame.shape[0])), 
                 line_color, line_thickness)  # 흰 선
        cv2.line(frame, hip_pos, 
                 (int(right_hip.x * frame.shape[1]), int(right_hip.y * frame.shape[0])), 
                 line_color, line_thickness)  # 흰 선

    
    else:
        # 랜드마크 정보가 없을 경우 NaN으로 채움
        frame_data.extend([None, None] * len(selected_landmarks))
        frame_data.extend([None, None])  # 목
        frame_data.extend([None, None])  # 엉덩이

    data.append(frame_data)

    # 시각화 (옵션)
#    if results.pose_landmarks:
#        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
    cv2.imshow('Pose Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 영상 및 리소스 해제
cap.release()
cv2.destroyAllWindows()
pose.close()

# 데이터를 CSV로 저장
df = pd.DataFrame(data, columns=columns)
df.to_csv(output_csv_path, index=False)

print(f"Joint 데이터를 {output_csv_path}에 저장했습니다.")

Joint 데이터를 ./BPE/training/id123_241206_165306.csv에 저장했습니다.


In [11]:
import os

#BPE 로 이동
os.chdir('C:\\Users\\USER\\yogafire\\BPE')

#BPE 안의 모델 파일 실행 / training 안에 방금 저장된 유저의 파일과 , rightyogajoint에 미리 저장해둔 요가 자세의 올바른 자세 joint 값과 비교 
output = os.popen(f"python inference_pair.py -v1 training/{filepath}.csv -v2 rightyogajoint/{select_yoga}.csv").read()

NameError: name 'filepath' is not defined

In [2]:
#점수 조정을 위한 함수 (수정가능)

def sigmoid(x):
    return 1 / (1 + np.exp(-x+1))

In [17]:
# 실행된 결과에서 score만 가져오기

list1 = output.split('\n')
score_str = list1[-2]

# score 점수 조정
score = ((round(sigmoid((float(score_str.split(' ')[-1])))*10)*10))
score

40

In [19]:
# 포멧팅이 아닌 그냥 테스트 데이터 삽입 (그냥 에시)

# import os

# output = os.popen(f"python BPE/inference_pair.py -v1 BPE/training/input12.csv -v2 BPE/training/input16.csv").read()
# output

'Building model\nC:\\Users\\USER\\yogafire\\BPE\\data\\model_epoch2.pth\nLoading model from C:\\Users\\USER\\yogafire\\BPE\\data\\model_epoch2.pth\nModel is ready\nsimilarity score: 0.7815807\n'